In [1]:
from tqdm import tqdm
import polars as pl
import json

In [2]:
krs = pl.read_parquet("/home/stef/enz_rxn_data/data/processed/pathway/stereo/known_reactions.parquet")
krs[2000:2005]

id,smarts,enzymes,reverse,db_ids
str,str,list[str],str,list[str]
"""32259cf17670326cfca47b7e776d7d…","""*C1CCC2C3CC=C4CC(=O)CCC4(C)C3C…","[""Q67477"", ""P26149"", … ""P9WQP6""]","""abd3928b7788ffa654eac3fe6dc71a…","[""RHEA:24078""]"
"""d8a38c1fc9830f3fb83ed2d3194330…","""*C1CCC2C3CC=C4CC(=O)CCC4(C)C3C…",[],"""6d70a93831f1b8f903d97203f83884…","[""RHEA:56882""]"
"""78faeefe8b57b4cd0ac2a1b6f1ff61…","""*C1CCC2C3CC=C4CC(=O)CCC4(C)C3C…","[""Q67477"", ""P26149"", … ""P9WQP6""]","""c67bc380817d37faeb2f07c5aff954…","[""RHEA:14710""]"
"""abd3928b7788ffa654eac3fe6dc71a…","""*C1CCC2C3CC=C4C[C@@H](O)CCC4(C…","[""Q67477"", ""P26149"", … ""P9WQP6""]","""32259cf17670326cfca47b7e776d7d…","[""RHEA:24077""]"
"""6d70a93831f1b8f903d97203f83884…","""*C1CCC2C3CC=C4C[C@@H](O)CCC4(C…",[],"""d8a38c1fc9830f3fb83ed2d3194330…","[""RHEA:56881""]"


In [3]:
krs = pl.read_parquet("/home/stef/enz_rxn_data/data/processed/pathway/stereo/known_compounds.parquet")
krs[2000:2005]

id,smiles,name,chebi_id,n_atoms
str,str,str,str,i32
"""a827abc37aa1b6a578babc72a3209e…","""CC(=O)N[C@@H](CSC(CC(=O)O)C(=O…","""N-acetyl-S-(2-succino)-L-cyste…","""CHEBI:144658""",18
"""73dcf81f094dd8838491726edffd5d…","""CC(=O)N[C@@H](CSC=O)C(=O)N[C@H…","""S-formylmycothiol""","""CHEBI:15735""",34
"""08be048130de3c3430c73dc5325438…","""CC(=O)N[C@@H](CSCO)C(=O)N[C@H]…","""S-(hydroxymethyl)mycothiol""","""CHEBI:61586""",34
"""76bc43f3648e022c4ed96d8996c6a8…","""CC(=O)N[C@@H](CSCc1ccccc1)C(=O…","""N-acetyl-S-benzyl-L-cysteine""","""CHEBI:194342""",17
"""29aa3645a3d81a8a7b799275d1ba02…","""CC(=O)N[C@@H](CSO)C(=O)O""","""N-acetyl-S-hydroxy-L-cysteine""","""CHEBI:194344""",11


In [19]:
krs = pl.read_parquet("/home/stef/enz_rxn_data/data/processed/pathway/known_reactions.parquet")
krs[2000:2005]

id,smarts,enzymes,reverse,db_ids
i32,str,list[str],i32,list[str]
2035,"""*C1CCC2C3CCC4C(C)(CCC(O)C4(C)C…","[""Q70VZ7"", ""Q96PD6"", … ""Q80W94""]",2039,"[""RHEA:60089""]"
2036,"""*C1CCC2C3CCC4C(C)(CO)C(O)CCC4(…","[""P52704""]",2031,"[""RHEA:60057""]"
2037,"""*C1CCC2C3CCC4C(C)(CO)C(O)CCC4(…","[""P52704""]",2029,"[""RHEA:60054""]"
2038,"""*C1CCC2C3CCC4C(C)C(=O)CCC4(C)C…","[""Q8BMJ2"", ""Q9P2J5"", ""Q5R614""]",2034,"[""RHEA:60086""]"
2039,"""*C1CCC2C3CCC4C(C)C(=O)CCC4(C)C…","[""Q70VZ7"", ""Q96PD6"", … ""Q80W94""]",2035,"[""RHEA:60090""]"


In [24]:
kes = pl.read_parquet("/home/stef/enz_rxn_data/data/processed/pathway/known_enzymes.parquet")
kes.head()

id,sequence,existence,reviewed,ec,organism,name
str,str,enum,enum,str,str,str
"""A0A009IHW8""","""MSLEQKKGADIISKILQIQNSIGKTTSPST…","""Evidence at protein level""","""reviewed""","""3.2.2.-; 3.2.2.6""","""Acinetobacter baumannii (strai…","""2' cyclic ADP-D-ribose synthas…"
"""A0A059TC02""","""MRSVSGQVVCVTGAGGFIASWLVKILLEKG…","""Evidence at protein level""","""reviewed""","""1.2.1.44""","""Petunia hybrida (Petunia)""","""Cinnamoyl-CoA reductase 1 (Ph-…"
"""A0A061I403""","""MPMASVIAVAEPKWISVWGRFLWLTLLSMA…","""Evidence at protein level""","""reviewed""","""2.7.7.108; 3.1.4.-""","""Cricetulus griseus (Chinese ha…","""Protein adenylyltransferase FI…"
"""A0A067XGX8""","""MALTATATTRGGSALPNSCLQTPKFQSLQK…","""Evidence at transcript level""","""reviewed""","""2.5.1.54""","""Petunia hybrida (Petunia)""","""Phospho-2-dehydro-3-deoxyhepto…"
"""A0A067XH53""","""MALSTNSTTSSLLPKTPLVQQPLLKNASLP…","""Evidence at transcript level""","""reviewed""","""2.5.1.54""","""Petunia hybrida (Petunia)""","""Phospho-2-dehydro-3-deoxyhepto…"


In [25]:
kes.filter(pl.col("existence").is_in(["Evidence at protein level", "Evidence at transcript level"]))

id,sequence,existence,reviewed,ec,organism,name
str,str,enum,enum,str,str,str
"""A0A009IHW8""","""MSLEQKKGADIISKILQIQNSIGKTTSPST…","""Evidence at protein level""","""reviewed""","""3.2.2.-; 3.2.2.6""","""Acinetobacter baumannii (strai…","""2' cyclic ADP-D-ribose synthas…"
"""A0A059TC02""","""MRSVSGQVVCVTGAGGFIASWLVKILLEKG…","""Evidence at protein level""","""reviewed""","""1.2.1.44""","""Petunia hybrida (Petunia)""","""Cinnamoyl-CoA reductase 1 (Ph-…"
"""A0A061I403""","""MPMASVIAVAEPKWISVWGRFLWLTLLSMA…","""Evidence at protein level""","""reviewed""","""2.7.7.108; 3.1.4.-""","""Cricetulus griseus (Chinese ha…","""Protein adenylyltransferase FI…"
"""A0A067XGX8""","""MALTATATTRGGSALPNSCLQTPKFQSLQK…","""Evidence at transcript level""","""reviewed""","""2.5.1.54""","""Petunia hybrida (Petunia)""","""Phospho-2-dehydro-3-deoxyhepto…"
"""A0A067XH53""","""MALSTNSTTSSLLPKTPLVQQPLLKNASLP…","""Evidence at transcript level""","""reviewed""","""2.5.1.54""","""Petunia hybrida (Petunia)""","""Phospho-2-dehydro-3-deoxyhepto…"
…,…,…,…,…,…,…
"""P80580""","""MKLYSFFNXRA""","""Evidence at protein level""","""reviewed""","""5.2.1.4""","""Klebsiella pneumoniae""","""Maleylpyruvate isomerase (EC 5…"
"""P83329""","""XXFENFLNANXAYVNLD""","""Evidence at protein level""","""reviewed""","""4.2.1.1""","""Streptococcus thermophilus""","""Putative carbonic anhydrase (E…"
"""P83855""","""GVYMEIGRCRXEAXRRRKEAV""","""Evidence at protein level""","""reviewed""","""4.6.1.1""","""Mus musculus (Mouse)""","""Putative sperm adenylate cycla…"


In [26]:
enzymes = set()
pairs = set()
proper_pairs = set()
proper_enzymes = set()
evidence_dict = dict(zip(kes["id"], kes["existence"]))
for row in tqdm(krs.iter_rows(named=True), total=len(krs)):
    rxn = row['smarts']
    for enz in row['enzymes']:
        enzymes.add(enz)
        evidence = evidence_dict.get(enz, "Unknown")
        
        if evidence in ["Evidence at protein level", "Evidence at transcript level"]:
            proper_pairs.add((rxn, enz))
            proper_enzymes.add(enz)
        
        pairs.add((rxn, enz))

len(pairs), len(proper_pairs)

100%|██████████| 28142/28142 [00:01<00:00, 20641.80it/s]


(518578, 164968)

In [27]:
print("total reacitions")
print(f"old: {len(old_rxns_w_enz)} vs new: {krs.shape[0]}")
print("\nreactions with enzymes")
print(f"old: {len(old_rxns_w_enz)} vs new: {new_rxns_w_enz.shape[0]}")

total reacitions
old: 18954 vs new: 28142

reactions with enzymes
old: 18954 vs new: 21034


In [28]:
print("number of enzymes")
print(f"old: {len(old_enz)} vs new: {len(enzymes)}")
print("\nnumber of proper enzymes")
print(f"old: {len(old_proper_enz)} vs new: {len(proper_enzymes)}")
print("\nnumber of pairs")
print(f"old: {len(old_pairs)} vs new: {len(pairs)}")
print("\nnumber of proper pairs")
print(f"old: {len(old_proper_pairs)} vs new: {len(proper_pairs)}")

number of enzymes
old: 180013 vs new: 188937

number of proper enzymes
old: 36531 vs new: 41238

number of pairs
old: 485570 vs new: 518578

number of proper pairs
old: 140234 vs new: 164968


In [29]:
kcs = pl.read_parquet("/home/stef/enz_rxn_data/data/processed/pathway/known_compounds.parquet")
kcs.tail()

id,smiles,name,chebi_id,n_atoms
i32,str,str,str,i32
10589,"""c1ccncc1""",""" pyridine""","""CHEBI:16227""",6
10590,"""c1cn[nH]c1""",""" pyrazole""","""CHEBI:17241""",5
10591,"""c1cncc(C2CCCN2)c1""",""" (S)-nornicotine""","""CHEBI:190184""",11
10592,"""c1ncc2c(n1)=NCCN=2""",""" 6,7-dihydropteridine""","""CHEBI:30156""",10
10593,"""c1ncc2c(n1)NCCN2""",""" 5,6,7,8-tetrahydropteridine""","""CHEBI:28889""",10


In [30]:
len(kcs)

10594